# Load occupancy data: RC-60-Mix

In [1]:
import numpy as np 
import pandas as pd

In [2]:
root_folder = 'preprocessed_data/'
f_x = root_folder + '058_5p_to_1p_X.txt'
f_y = root_folder + '058_5p_to_1p_y.txt'
data = pd.read_csv(f_x, delimiter=' ', header = None)
labels = pd.read_csv(f_y, delimiter=' ', header = None)
print(data.shape, labels.shape)

(4340, 5976) (4340, 1)


# Train 1d cnn on RC-60-Mix

In [3]:
from das_lib.models import cnn1d
from das_lib.funs import split_train_test_2d
import tensorflow as tf

2023-07-10 11:32:13.430447: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [4]:
import random

def split_train_test_1d(data, labels, frac=.8, index=None):
    x_train = data.sample(frac=frac) if index is None else data.iloc[index]
    x_test = data.drop(x_train.index)
    y_train = labels.iloc[x_train.index]
    y_test = labels.drop(x_train.index)
    return x_train, x_test, y_train, y_test, x_train.index

In [5]:
x_train, x_test, y_train, y_test, x_train_index = split_train_test_1d(data, labels, frac=.67, index=None)
x_train = np.expand_dims(x_train,-1)
x_test = np.expand_dims(x_test,-1)
y_train = tf.keras.utils.to_categorical(y_train-1, num_classes=5)
y_test = tf.keras.utils.to_categorical(y_test-1, num_classes=5)


In [6]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((2908, 5976, 1), (1432, 5976, 1), (2908, 5), (1432, 5))

In [7]:
m = cnn1d( (x_train.shape[1],1), y_train.shape[1])
print(m.summary())

2023-07-10 11:32:47.931225: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-10 11:32:50.755705: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38251 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:2f:00.0, compute capability: 8.0


Model: "1dcnn_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 5976, 1)]         0         
                                                                 
 conv1d (Conv1D)             (None, 2957, 16)          1040      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 369, 16)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 169, 32)           16416     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 21, 32)           0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 3, 16)             

In [8]:
H = m.fit(x_train, y_train, epochs=1000, batch_size=64, validation_data=(x_test, y_test), verbose=False)
m.evaluate(x_test, y_test, batch_size=64, verbose=True)

2023-07-10 11:32:55.618509: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8101
2023-07-10 11:33:01.281638: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


23/23 [==============================] - 0s 4ms/step - loss: 2.8320 - accuracy: 0.8128


[2.8319950103759766, 0.8128491640090942]

In [9]:
checkpoint_folder = 'training_results'
model_name = '5way_1d'
checkpoint_path = checkpoint_folder+'/'+'{0}'.format(model_name)
m.save(checkpoint_path)

INFO:tensorflow:Assets written to: training_results/5way_1d/assets


INFO:tensorflow:Assets written to: training_results/5way_1d/assets


In [ ]:
from das_lib.evaluation import save_training_history
f_name = 'performance/5way_1d.json'
save_training_history(f_name, H)

# Read the trained model

In [2]:
import tensorflow as tf
checkpoint_folder = 'training_results'
model_name = '5way_1d'
checkpoint_path = checkpoint_folder+'/'+'{0}'.format(model_name)
m = tf.keras.models.load_model(checkpoint_path)


2023-07-10 15:45:23.608285: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-10 15:45:45.379271: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-10 15:45:47.694316: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38251 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:2f:00.0, compute capability: 8.0


# Validate the trained model on Ind. RC-60-1p

In [3]:
import numpy as np

In [4]:
def expand_data(data, label):
    data= np.expand_dims(data,-1)
    label = np.array([label]*len(data))
    label = tf.keras.utils.to_categorical(label-1, num_classes=5)
    return data, label

In [5]:
from utils.sampling import extract_car_data
root_folder = 'preprocessed_data/'
f_x = root_folder+'026_X.txt' 
f_y = root_folder+'026_y.txt'
car_label = 2 # in speed data (023-027), car 2 is label = 2
car2_data =  extract_car_data(f_x, f_y, car_label)

In [6]:
label = 5
car2_1p_data, car2_1p_label_new = expand_data(car2_data, label)
print(car2_1p_data.shape, car2_1p_label_new.shape)
m.evaluate(car2_1p_data, car2_1p_label_new)

(451, 5976, 1) (451, 5)


2023-07-10 15:45:53.533685: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8101


15/15 [==============================] - 9s 5ms/step - loss: 36.1042 - accuracy: 0.2971


2023-07-10 15:45:59.644064: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


[36.104225158691406, 0.29711753129959106]

In [7]:
from das_lib.evaluation import save_wrong_indices
f_name = 'performance/wrong_indices/5way_1d_1p.json'
save_wrong_indices(f_name, m, car2_1p_data, car2_1p_label_new)

15/15 [==============================] - 0s 3ms/step


# Validate the trained model on Ind. RC-60-5p

In [8]:
root_folder = 'preprocessed_data/'
f_x = root_folder+'058_5p_X.txt' 
f_y = root_folder+'058_5p_y.txt'
car_label = 1 #old label, direction west => label=1
car2_data = extract_car_data(f_x, f_y, car_label)

In [9]:
label = 1 #new label, 5p in car 2 = > label = 1 
car2_5p_data, car2_5p_label_new = expand_data(car2_data, label)
print(car2_5p_data.shape, car2_5p_label_new.shape)

(291, 5976, 1) (291, 5)


In [10]:
m.evaluate(car2_5p_data, car2_5p_label_new)

10/10 [==============================] - 0s 4ms/step - loss: 66.8639 - accuracy: 0.0997


[66.86390686035156, 0.0996563583612442]

In [11]:
f_name = 'performance/wrong_indices/5way_1d_5p.json'
save_wrong_indices(f_name, m, car2_5p_data, car2_5p_label_new)

10/10 [==============================] - 0s 3ms/step
